In [1]:
import imageio.v3 as iio
import requests
import numpy as np
import os
from datetime import datetime
import sys  
import colorsys
import csv

In [2]:
# Make api call to flickr
# for some reason, flickr key isn't found (KeyError)

flickr_api_url = "https://www.flickr.com/services/rest/?method=flickr.photos.search&api_key=f8ac40eee0aacfa3be6dee0cbac63930" +\
            "&tags=sunset" +\
            "&text=sunset%20landscape" +\
            "&content_type=1" +\
            "&has_geo=1" +\
            "&sort=interestingness-desc" +\
            "&extras=date_taken%2C+geo%2C+url_o" +\
            "&format=json&nojsoncallback=1"
            # "&is_getty=1" +\             

flickr_res = requests.get(flickr_api_url)

if(flickr_res.status_code < 200 or flickr_res.status_code >= 300): 
    print('Unsuccessful request. Error code: {}'.format(flickr_res.status_code))
    sys.exit()
else:
    print("Flickr API call was successful.")
    flickr_res = flickr_res.json()

Flickr API call was successful.


In [3]:
# write csv header
fields = ['calc_score', 'humidity', 'cloud_cover', 'visibility', 'user_score', 'lat', 'long', 'date_time', 'image_url']

with open('./data.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    row = ['a', 'b', 'c']
    writer.writerow(fields)

In [4]:
# Process multiple images at once
VISUALCROSSING_API_KEY = os.environ["VISUALCROSSING_API_KEY"]
DATE_TIME_FORMAT_STRING = "%Y-%m-%d %H:%M:%S"
SAMPLE_SCALE = 5

def score_sunset(image):
    sunset_pixel_counter = 0
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            rVal = image[i,j,0]
            gVal = image[i,j,1]  
            bVal = image[i,j,2]
            hVal = colorsys.rgb_to_hls(rVal, gVal, bVal)[0]*360
            if(hVal <= 45): # oranges are weighted less
                sunset_pixel_counter += 0.7
            elif(hVal >= 270): # reds and magentas are weighted fully
                sunset_pixel_counter +=1

    total_image_pixels = image.shape[1] *  image.shape[0]
    sunset_score = ((sunset_pixel_counter / total_image_pixels) * 10)
    sunset_score = round(sunset_score, 3)
    return sunset_score

f = open('./data.csv', 'a', newline='')
writer = csv.writer(f)

print("Length of Flickr API response: ", len(flickr_res['photos']['photo']))

for i in range(10, 100):
    # Get and load pic from flickr res
    photo = flickr_res['photos']['photo'][i]
    imgUrl = f'https://live.staticflickr.com/{photo["server"]}/{photo["id"]}_{photo["secret"]}_b.jpg'
    pic = iio.imread(imgUrl)
    
    # GET WEATHER DATA FOR IMAGE
    # nearest hour of sunset for picture
    date_taken = datetime.strptime(photo["datetaken"], DATE_TIME_FORMAT_STRING)
    nearest_sunset_hour = date_taken.hour
    if date_taken.minute > 30:
        nearest_sunset_hour +=  1
    
    # make api call for picture
    visualcrossing_history_api_url = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/history?" +\
                            "&aggregateHours=1" +\
                            "&startDateTime=" + str(date_taken.date()) +\
                            "&endDateTime=" + str(date_taken.date())  +\
                            "&unitGroup=metric" +\
                            "&contentType=json" +\
                            "&dayStartTime=" + str(nearest_sunset_hour) + ":0:00" +\
                            "&dayEndTime=" + str(nearest_sunset_hour + 1) + ":0:00" +\
                            "&location=" + photo["latitude"] + "%2C%20" + photo["longitude"] +\
                            "&key=" + VISUALCROSSING_API_KEY
    weather_res = requests.get(visualcrossing_history_api_url)

    if(weather_res.status_code != 200): 
        print('Unsuccessful request. Error code: {}'.format(weather_res.status_code))
        sys.exit()
    else:
        # print('Visual Crossing API call was successful for index ', i)
        weather_res = weather_res.json()
    
    # get weather data of picture
    try:
        nearest_hour_data = weather_res['locations'][photo["latitude"] + ", " + photo["longitude"]]['values'][0]
        humidity = nearest_hour_data['humidity']
        cloud_cover = nearest_hour_data['cloudcover']
        visibility = nearest_hour_data['visibility']
    except KeyError:
        if(weather_res['errorCode'] == -994): # api call quota hit
            print(weather_res)
            break
        else:
            # handles errorCode 999 (weird stuff)
            print(weather_res)
        
    # prevent images with empty weather data from being written
    if(humidity == None or cloud_cover == None or visibility == None):
        print("image" , i, "doesn't have all data:")
        print(humidity, cloud_cover, visibility)
        continue

    # ANALYZE PHOTO
    length = pic.shape[0]
    width = pic.shape[1]
    sys_sample_pic = pic[0:length:SAMPLE_SCALE, 0:width:SAMPLE_SCALE, :]
    calc_score = score_sunset(sys_sample_pic)

    # WRITE TO CSV
    row = [calc_score, humidity, cloud_cover, visibility, "", photo["latitude"], photo["longitude"], date_taken, imgUrl]
    writer.writerow(row)

f.close()

Length of Flickr API response:  250


c:\Python310\lib\colorsys.py:78: RuntimeWarning: overflow encountered in ubyte_scalars
  sumc = (maxc+minc)
c:\Python310\lib\colorsys.py:86: RuntimeWarning: divide by zero encountered in double_scalars
  s = rangec / (2.0-sumc)
c:\Python310\lib\colorsys.py:84: RuntimeWarning: divide by zero encountered in ubyte_scalars
  s = rangec / sumc


image 25 doesn't have all data:
None None None
{'executionTime': -1, 'sessionId': '', 'errorCode': 999, 'message': 'No weather stations found within 50.0mi of location 21.987053, -69.706850.'}
image 49 doesn't have all data:
72.38 2.1 None
{'executionTime': -1, 'sessionId': '', 'errorCode': 999, 'message': 'No weather stations found within 50.0mi of location 41.518988, -3.531160.'}
image 72 doesn't have all data:
65.91 0.0 None
image 91 doesn't have all data:
None None None
image 93 doesn't have all data:
76.1 0.0 None
image 97 doesn't have all data:
66.37 0.0 None
